# Step 1: 構造化データのVector DB登録

本ステップでは、RAGに必要な類似検索で利用するベクトルデータベースの環境を整えることを目的に、構造化データとして用意したExcelファイルをベクトル化してベクトルデータベースに保存する過程を経験します。
- 構造化データのExcelファイルをPandas DataFrameに読み込みます
- 今回はExcelの1行をベクトルデータベースの1行として登録します（Excelで管理している行より細かくチャンキングは行いません）
- ベクトルデータベースに登録するため、DataFrameからメタデータとコンテンツの構造から成るLangChainのDocument形式に変換します
- Hugging FaceからEmbedding Modelを読み込み、エンベディング（ベクトル化）しながらベクトルデータベースに登録します
![Step1](../image/rag-overview-step1.png)

**用意している構造化データについて**

本ステップを体験するために、構造化データを用意しておりますが、事前に以下サイトから講座情報を収集してExcelファイルに保存しています。
- サイト名：東京都 東京リカレントナビ
- URL：https://www.recurrent-navi.metro.tokyo.lg.jp

| ![Step1](../image/tyo_recurrent_navi-top.png) | ![Step1](../image/tyo_recurrent_navi-seminar.png) |
|:---:|:---:|
| 図. トップページ | 図. 講座情報ページ |


## 0. 事前準備

### 共通処理/定数定義
全ステップで共通して使用する定数を宣言してファイルに書き出します。

In [ ]:
%%writefile mylib/myconstant.py
VDB_HOST = "127.0.0.1"
VDB_PORT = 19530
VDB_USER = "root"
VDB_PASS = "Milvus"

処理時間を把握するためのバナークラスを定義してクラスファイルに書き出します。

In [ ]:
%%writefile mylib/MyBanner.py
import time
import datetime

class MyBanner:
    s_time = 0
    @staticmethod
    def start():
    	MyBanner.s_time = time.perf_counter()
    	print("### START (%s) ###########" % (datetime.datetime.now()))

    @staticmethod
    def passing(message):
        n_time = time.perf_counter()
        print(">>> %s (lap time = %.2f sec)" % (message, n_time - MyBanner.s_time))

    @staticmethod
    def finish():
    	e_time = time.perf_counter()
    	print("### FINISH (%s > duration = %.2f sec) ###########" % (datetime.datetime.now(), e_time - MyBanner.s_time))

全ステップで共通して使用する定数とバナークラスを読み込み、本ステップで使用する定数を宣言します。

In [ ]:
from mylib import myconstant
from mylib.MyBanner import MyBanner

EXCEL_FILE = '../data/recurrent_navi_tyo.xlsx'
META_FIELDS = {"No.": "id", "施設の分類": "category"}
META_FIELDS = {}

### パッケージインストール
本ステップの処理で依存するパッケージをインストールします。

In [ ]:
MyBanner.start()

!python -V
!pip install pandas
!pip install openpyxl
!pip install langchain
!pip install langchain-huggingface
!pip install langchain-milvus
!pip install sentence-transformers

!pip install ipywidgets
!pip install urllib3==1.26.20

MyBanner.finish()

### import
本ステップの処理で依存するモジュールを読み込みます。

In [ ]:
MyBanner.start()

import pandas as pd
import os
import json
from langchain_core.documents import Document

MyBanner.finish()

## 1. 構造化データ読込

### 【準備】ExcelデータDF読込
Excelファイルの全シートをDF(Pandas DataFrame)形式で読み込みます。

In [ ]:
MyBanner.start()

# load from all sheets and store them in df_list.
file_path = EXCEL_FILE
print(f"{file_path=}")

df_list = []
for sheet_name in pd.ExcelFile(file_path).sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df_list.append(df)

# find the collection name from an excel file name
collection_name = os.path.splitext(os.path.basename(file_path))[0]
print(f"{collection_name=}")

MyBanner.finish()

## 2. Chunking

### 【準備】Cleansing
ベクトルデータベースで類似検索の効率が上がるようにデータを整えます。
- 'id'と'category'をメタ情報とするため、メタ情報として扱うカラムを該当のカラム名に置き換えます

In [ ]:
MyBanner.start()

for index in range(len(df_list)):
    df = df_list[index]
    df = df.rename(columns=META_FIELDS)
    df_list[index] = df
    # Show the first 5 rows for each sheet
    print(f"\nExcel sheet name: {sheet_name}")
    display(df.head())

MyBanner.finish()

### 【準備】JSON変換とメタ情報抽出
LangChain Document形式の格納に向けてJSON形式に変換しメタ情報を抽出します。
- DataFrameの行単位にセル列をフィールドとしたJSON形式に変換します
- 1つのExcelファイルで全てのシートを集約して、セル情報のListを生成します
- 'id'と’category'列を抜き出し、メタ情報のListを生成します

In [ ]:
MyBanner.start()

json_meta_list=[]
json_content_list=[]

# Iterate through each sheet
for df in df_list:
    json_meta_string =  json.loads(df[['id', 'category']].to_json(orient='records', force_ascii=False), parse_int=str)
    json_doc_string = json.loads(df.to_json(orient='records', force_ascii=False))
    json_meta_list.extend(json_meta_string)
    json_content_list.extend(json_doc_string)

print(f"{len(json_meta_list)=}")
print(f"{len(json_content_list)=}")

# Show data for inspection
print("\n* Meta-data (the first 5 rows):")
for index, item in enumerate(json_meta_list[0:5]):
    print(index + 1, item)

print("\n* Vectorization data (the first 5 rows):")
for index, item in enumerate(json_content_list[0:5]):
    print(index + 1, item)

MyBanner.finish()

### 【処理】LangChain Doc変換
ベクトルデータベースの登録向けにメタ情報とセル情報をLangChain Document形式に変換します。
- DocumentのListをjson_meta_listとjson_doc_listから作成します
- 1要素のDocumentでmetadataは、json_meta_listの一行分のjsonで構成します
- 1要素のDocumentでpage_contentはjson_doc_listの一行分のjsonで構成します

In [ ]:
MyBanner.start()

docs = []
for content_str, meta_str in zip(json_content_list, json_meta_list):
    docs.append(Document(metadata=meta_str, page_content=json.dumps(content_str, ensure_ascii=False))) 

# Show data for inspection (the first 2 rows)
print(f"{len(docs)=}")
[(print(f"{doc=}\n")) for doc in docs[0:2]]

MyBanner.finish()

## 3. Vector DB登録

### 【定義】Embedding Class
Embedding ModelをHugging Faceから取得して、メモリに読み込んで管理するためのクラスを定義してクラスファイルに書き出します。

In [ ]:
%%writefile mylib/MyEmbedding.py
from langchain_huggingface import HuggingFaceEmbeddings

class MyEmbedding:
    @staticmethod
    def get_model(model_name = "intfloat/multilingual-e5-large"):
        model = HuggingFaceEmbeddings(model_name = model_name)
        return model

In [ ]:
MyBanner.start()
from mylib.MyEmbedding import MyEmbedding

embeddings = MyEmbedding.get_model()
print(f"{embeddings=}")

MyBanner.finish()

### 【定義】MyCustomRetriever Class
類似スコア検索を実現するための独自処理を実装したRetrieverを定義してクラスファイルに書き出します。

In [ ]:
%%writefile mylib/MyCustomRetriever.py
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.callbacks.manager import (CallbackManagerForRetrieverRun)
from typing import List
from langchain_core.documents import Document

class MyCustomRetriever(VectorStoreRetriever):
    def _get_relevant_documents(
        self, query: str, *,
        run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        top_k = self.search_kwargs.get("k", 4)
        docs_and_similarities = self.vectorstore.similarity_search_with_score(query, k=top_k)      
        threshold = self.search_kwargs.get("score_threshold", 0)
        return [doc for doc, score in docs_and_similarities if score >= threshold and score <= 1]
    

### 【定義】MyMilvus Class
ベクトルデータベースとして使う「Milvus」を管理するためのクラスを定義してクラスファイルに書き出します。

In [ ]:
%%writefile mylib/MyMilvus.py
from langchain_milvus import Milvus
from pymilvus import MilvusClient
from mylib.MyCustomRetriever import MyCustomRetriever

class MyMilvus:

    def __init__(self, host, port, user, password, embeddings):
        self.connection_args = self.__get_connect_args(
            host, port, user, password)
        self.embeddings = embeddings
        db_name = "default"
        self.client = MilvusClient(
            uri = "http://%s:%d" % (host, port),
            token = "%s:%s" % (user, password),
            db_name = db_name)

    def get_connection_args(self):
        return self.connection_args

    def get_collections(self):
        collections = self.client.list_collections()
        return collections

    # connect to the collection
    def connect(self, collection_name):
        collection = Milvus(
            self.embeddings,
            connection_args = self.connection_args,
            collection_name = collection_name
        )
        return collection

    def from_documents(self, docs, collection_name):
        index_params = self.__get_index_params()
        collection = Milvus.from_documents(
            docs,
            self.embeddings,
            connection_args = self.connection_args,
            collection_name = collection_name,
            index_params = index_params,
            drop_old = True, # If adding data, you should set False here.
        )
        return collection
    
    def get_retriever(self, collection, k = None, score = None):
        if (k is None):
            retriever = collection.as_retriever()
        else:
            if (score is None):
                retriever = collection.as_retriever(search_kwargs={"k": k})
            else:
#                retriever = collection.as_retriever(
#                    search_type="similarity_score_threshold",
#                    search_kwargs={"k": k, "score_threshold": score})
                retriever = MyCustomRetriever(
                    vectorstore = collection,
                    search_kwargs={"k": k, "score_threshold": score})
        return retriever

    def __get_connect_args(self, host, port, user, password):
        args ={
            'uri': "http://%s:%d" % (host, port),
            'token': "%s:%s" % (user, password)
        }
        return args

    def __get_index_params(self):
        params = {
            "metric_type": "COSINE", # Cosine Similarity
            "index_type": "HNSW", 
            "params": { "M": 16, "efConstruction": 200, "efSearch": 16}
        }
        return params


### 【処理】EmbeddingとVDB挿入
エンベディングしながらベクトルデータベースにデータを登録します。

In [ ]:
MyBanner.start()

from mylib.MyMilvus import MyMilvus

vector_db = MyMilvus(\
    myconstant.VDB_HOST, myconstant.VDB_PORT,\
    myconstant.VDB_USER, myconstant.VDB_PASS, embeddings)
print(f"{vector_db=}")

# Insert into Vector DB while embedding immediately after connecting to it
docstore = vector_db.from_documents(docs, collection_name)
print(f"{len(docs)=}")
print(f"{collection_name=}")
print(f"{docstore=}")

MyBanner.finish()

## 4. Vector DB登録データ確認

### 確認方法1: Python言語
ベクトルデータベースに登録されたデータをPythonのソースコードで確認します。

In [ ]:
MyBanner.start()

from pymilvus import MilvusClient
import pandas as pd

pk_list = docstore.get_pks(expr = "pk > 0")
connection_args = vector_db.get_connection_args()
client = MilvusClient(uri = connection_args['uri'], token = connection_args['token'])
res = client.get(
    collection_name = collection_name,
    ids=pk_list
)

for i, doc_rec in enumerate(res):
    res[i]['vector'] = "["+", ".join(map(str, doc_rec['vector']))+"]"
    res[i]['pk'] = str(doc_rec['pk'])

df_s = pd.DataFrame.from_dict(res).reindex(columns = ['id', 'category', 'text', 'pk', 'vector'])
display(df_s)

MyBanner.finish()

### 確認方法2: Attu(Web UI)
ベクトルデータベースに登録されたデータをWeb UIのAttuを使って確認します。

MilvusのWeb UI管理ツールの「Attu」をコンテナ（コンテナ名：milvus-attu）で起動しているので、ツールにアクセスして実際にベクトル化して登録されたデータを目視して確認します。
- URL: http://localhost:8000/

主要な画面のキャプチャーと概要説明
| ![ログイン](./../image/attu-login.png) | ![データベース](./../image/attu-dbs.png) |
|:---:|:---:|
| 図1. ログイン | 図2. データベース一覧 |
| ![コレクション](./../image/attu-collections.png) | ![レコード](./../image/attu-records.png) |
| 図3. コレクション一覧 | 図4. コレクション詳細 |

- 図1. ログイン
	- アクセスすると画面にMilvus-Address、Milvus Databaseなどがデフォルトで入力されています
	- 値は変更せず、全てデフォルトのままでConnectボタンを押下してデータベース一覧へ遷移して利用を開始します
- 図2. データベース一覧
	- Milvusに登録されているデータベースが一覧表示されています
	- 本ステップではdefaultデータベースのコレクションにデータを登録しているので、データベース一覧より該当のデータベースを押下してコレクション一覧に遷移します
- 図3. コレクション一覧
	- defaultデータベースに登録されているコレクションが一覧表示されています
	- 本ステップではrecurrent_navi_tyoコレクションにデータを登録しているので、コレクション一覧より該当のコレクションを押下してコレクション詳細に遷移します
- 図4. コレクション詳細
	- SchemaからPropertiesまで7つのタブで構成されています
	- Dataタブを押下すると、本ステップで実際に登録したデータのベクトル値を含めた登録状態を目視できます

## 5. 本ステップを終えて

ここまでの手順で構造化データをベクトルデータベースに登録する過程を経験しました。次のステップではベクトルデータベースに登録されているデータで類似検索の実行を経験します。
- 次のStep ≫ [Step 2: Vector DBで類似検索](./rag-step02-search_from_vectordb.ipynb)
- 今のStep ≫ Step 1: 構造化データのVector DB登録